In [1]:
import soundfile as sf
import os, librosa
import numpy as np
import torch
import torch.nn as nn
import random
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, BertTokenizer, BertModel
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import timeit
# from pynvml import *

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
asr_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to(DEVICE)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
# batch data preparation
path = "/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/IEMOCAP/Data"
os.chdir(path)
names_train = []
names_test = []
emolabels_train = []
emolabels_test = []

train_asr_hidden0 = []
train_asr_hidden1 = []
train_asr_hidden2 = []
train_asr_hidden3 = []
train_asr_hidden4 = []
train_asr_hidden5 = []
train_asr_hidden6 = []
train_asr_hidden7 = []
train_asr_hidden8 = []
train_asr_hidden9 = []
train_asr_hidden10 = []
train_asr_hidden11 = []
train_asr_hidden12 = []

test_asr_hidden0 = []
test_asr_hidden1 = []
test_asr_hidden2 = []
test_asr_hidden3 = []
test_asr_hidden4 = []
test_asr_hidden5 = []
test_asr_hidden6 = []
test_asr_hidden7 = []
test_asr_hidden8 = []
test_asr_hidden9 = []
test_asr_hidden10 = []
test_asr_hidden11 = []
test_asr_hidden12 = []

with open('emo_name_trans_5500.txt', 'r') as f:
    files = f.readlines()
    files = np.array(files)
    indices = np.arange(len(files))
    random.shuffle(indices)
    files[np.arange(len(files))] = files[indices]
    for line in files:
        emotion = line.split(" ")[0]
        file_name = line.split(" ")[1]
        
        if emotion == "Ang":
            emotion_label = 0
        elif emotion == "Hap":
            emotion_label = 1
        elif emotion == "Neu":
            emotion_label = 2
        elif emotion == "Sad":
            emotion_label = 3
        
        audio_input, sr = sf.read(file_name)
        inputs = processor(audio_input, sampling_rate=16000, return_tensors='pt').input_values.to(DEVICE)
        hiddens = asr_model(inputs, output_hidden_states=True).hidden_states
        hidden0 = hiddens[0].detach()[0].mean(dim=0)
        hidden1 = hiddens[1].detach()[0].mean(dim=0)
        hidden2 = hiddens[2].detach()[0].mean(dim=0)
        hidden3 = hiddens[3].detach()[0].mean(dim=0)
        hidden4 = hiddens[4].detach()[0].mean(dim=0)
        hidden5 = hiddens[5].detach()[0].mean(dim=0)
        hidden6 = hiddens[6].detach()[0].mean(dim=0)
        hidden7 = hiddens[7].detach()[0].mean(dim=0)
        hidden8 = hiddens[8].detach()[0].mean(dim=0)
        hidden9 = hiddens[9].detach()[0].mean(dim=0)
        hidden10 = hiddens[10].detach()[0].mean(dim=0)
        hidden11 = hiddens[11].detach()[0].mean(dim=0)
        hidden12 = hiddens[12].detach()[0].mean(dim=0)

        if "Ses05" not in file_name:
            names_train.append(file_name)
            emolabels_train.append(emotion_label)
            train_asr_hidden0.append(hidden0.view(1, 768).cpu().numpy())
            train_asr_hidden1.append(hidden1.view(1, 768).cpu().numpy())
            train_asr_hidden2.append(hidden2.view(1, 768).cpu().numpy())
            train_asr_hidden3.append(hidden3.view(1, 768).cpu().numpy())
            train_asr_hidden4.append(hidden4.view(1, 768).cpu().numpy())
            train_asr_hidden5.append(hidden5.view(1, 768).cpu().numpy())
            train_asr_hidden6.append(hidden6.view(1, 768).cpu().numpy())
            train_asr_hidden7.append(hidden7.view(1, 768).cpu().numpy())
            train_asr_hidden8.append(hidden8.view(1, 768).cpu().numpy())
            train_asr_hidden9.append(hidden9.view(1, 768).cpu().numpy())
            train_asr_hidden10.append(hidden10.view(1, 768).cpu().numpy())
            train_asr_hidden11.append(hidden11.view(1, 768).cpu().numpy())
            train_asr_hidden12.append(hidden12.view(1, 768).cpu().numpy())
        elif "Ses05" in file_name:
            names_test.append(file_name)
            emolabels_test.append(emotion_label)
            test_asr_hidden0.append(hidden0.view(1, 768).cpu().numpy())
            test_asr_hidden1.append(hidden1.view(1, 768).cpu().numpy())
            test_asr_hidden2.append(hidden2.view(1, 768).cpu().numpy())
            test_asr_hidden3.append(hidden3.view(1, 768).cpu().numpy())
            test_asr_hidden4.append(hidden4.view(1, 768).cpu().numpy())
            test_asr_hidden5.append(hidden5.view(1, 768).cpu().numpy())
            test_asr_hidden6.append(hidden6.view(1, 768).cpu().numpy())
            test_asr_hidden7.append(hidden7.view(1, 768).cpu().numpy())
            test_asr_hidden8.append(hidden8.view(1, 768).cpu().numpy())
            test_asr_hidden9.append(hidden9.view(1, 768).cpu().numpy())
            test_asr_hidden10.append(hidden10.view(1, 768).cpu().numpy())
            test_asr_hidden11.append(hidden11.view(1, 768).cpu().numpy())
            test_asr_hidden12.append(hidden12.view(1, 768).cpu().numpy())

f.close()

emolabels_train = torch.tensor(emolabels_train)
emolabels_test = torch.tensor(emolabels_test)
train_asr_hidden0 = torch.tensor(train_asr_hidden0).to(DEVICE)
train_asr_hidden1 = torch.tensor(train_asr_hidden1).to(DEVICE)
train_asr_hidden2 = torch.tensor(train_asr_hidden2).to(DEVICE)
train_asr_hidden3 = torch.tensor(train_asr_hidden3).to(DEVICE)
train_asr_hidden4 = torch.tensor(train_asr_hidden4).to(DEVICE)
train_asr_hidden5 = torch.tensor(train_asr_hidden5).to(DEVICE)
train_asr_hidden6 = torch.tensor(train_asr_hidden6).to(DEVICE)
train_asr_hidden7 = torch.tensor(train_asr_hidden7).to(DEVICE)
train_asr_hidden8 = torch.tensor(train_asr_hidden8).to(DEVICE)
train_asr_hidden9 = torch.tensor(train_asr_hidden9).to(DEVICE)
train_asr_hidden10 = torch.tensor(train_asr_hidden10).to(DEVICE)
train_asr_hidden11 = torch.tensor(train_asr_hidden11).to(DEVICE)
train_asr_hidden12 = torch.tensor(train_asr_hidden12).to(DEVICE)
test_asr_hidden0 = torch.tensor(test_asr_hidden0).to(DEVICE)
test_asr_hidden1 = torch.tensor(test_asr_hidden1).to(DEVICE)
test_asr_hidden2 = torch.tensor(test_asr_hidden2).to(DEVICE)
test_asr_hidden3 = torch.tensor(test_asr_hidden3).to(DEVICE)
test_asr_hidden4 = torch.tensor(test_asr_hidden4).to(DEVICE)
test_asr_hidden5 = torch.tensor(test_asr_hidden5).to(DEVICE)
test_asr_hidden6 = torch.tensor(test_asr_hidden6).to(DEVICE)
test_asr_hidden7 = torch.tensor(test_asr_hidden7).to(DEVICE)
test_asr_hidden8 = torch.tensor(test_asr_hidden8).to(DEVICE)
test_asr_hidden9 = torch.tensor(test_asr_hidden9).to(DEVICE)
test_asr_hidden10 = torch.tensor(test_asr_hidden10).to(DEVICE)
test_asr_hidden11 = torch.tensor(test_asr_hidden11).to(DEVICE)
test_asr_hidden12 = torch.tensor(test_asr_hidden12).to(DEVICE)

In [26]:
indices_train = []
for i in range(len(names_train)):
    indices_train.append(i)

In [5]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
#         self.pool = nn.MaxPool1d(2)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=64,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        self.attn = nn.MultiheadAttention(64, 16, batch_first=True)
        self.dense = nn.Linear(64, 16)
        self.acti = nn.ReLU()
        self.out= nn.Linear(16, 4)

    def forward(self, x):
#         x = self.pool(x)
        self.lstm.flatten_parameters()
        x, _ = self.lstm(x)
#         x, _ = self.lstm2(x)
        x, _ = self.attn(x,x,x)
        x = x.mean(dim=1)
        x = self.dense(x)
        
        res = self.acti(x)
        emotion = self.out(res)
        return emotion

In [42]:
ser_model = NeuralNet().to(DEVICE)
asr_model = asr_model.to(torch.float)
params = ser_model.parameters()
optimizer = torch.optim.Adam(params, lr=1e-4, weight_decay=1e-5)

func = nn.CrossEntropyLoss()

epochs = 100
epoch = 0
batch_size = 64

# training
while epoch < epochs:
    torch.cuda.empty_cache()
    start = timeit.default_timer()
    print("-----epoch: ", epoch, "-----")
    loss_list_train = []
    loss_list_test = []
    predictions_train = []
    predictions_test = []

    print('--training begins--')
    ser_model.train()
    j = 0
    while j < len(names_train)/batch_size:
        if (j+1)*batch_size > len(names_train):
            emolabels = torch.tensor(emolabels_train[j*batch_size: ]).to(DEVICE)
            indices = indices_train[j*batch_size: ]
            asr_feats = train_asr_hidden12[j*batch_size: ].to(DEVICE)
        else:
            emolabels = torch.tensor(emolabels_train[j*batch_size:(j+1)*batch_size]).to(DEVICE)
            indices = indices_train[j*batch_size:(j+1)*batch_size]
            asr_feats = train_asr_hidden12[j*batch_size:(j+1)*batch_size].to(DEVICE)

# ser loss
        pred = ser_model(asr_feats)
        train_ser_loss = func(pred, torch.tensor(emolabels))
        for i in pred:
            predictions_train.append(i.detach().cpu().numpy())
        j += 1

# backprop
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(params, 1.0)
        train_ser_loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
    print('--training ends--')


# testing
# extract features
    print('--testing begins--')
    ser_model.eval()

# ser loss
    pred = ser_model(test_asr_hidden12)
    test_ser_loss = func(pred, torch.tensor(emolabels_test).to(DEVICE))
    for i in pred:
        predictions_test.append(i.detach().cpu().numpy())
    torch.cuda.empty_cache()
#     k += 1
        
# compute performance for each epoch
    predictions_train = np.array(predictions_train)
    predictions_test = np.array(predictions_test)
    emolabels_train = np.array(emolabels_train)
    emolabels_test = np.array(emolabels_test)
    predictions_train = [np.argmax(p) for p in predictions_train]
    acc_train = accuracy_score(emolabels_train, predictions_train)
    predictions_test = [np.argmax(p) for p in predictions_test]
    acc_test = accuracy_score(emolabels_test, predictions_test)
 
    print('Epoch:', epoch, '|train_ser_loss: %.4f' %train_ser_loss, '|train_accuracy: %.4f' %acc_train, '|test_ser_loss: %.4f' %test_ser_loss, '|test_accuracy: %.4f' %acc_test)
    print('test: \n', confusion_matrix(emolabels_test, predictions_test))
    print(classification_report(emolabels_test, predictions_test))

    epoch += 1
    print('---testing ends---')
    
    stop = timeit.default_timer()
    print('Time: ', stop - start) 

-----epoch:  0 -----
--training begins--


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


--training ends--
--testing begins--
Epoch: 0 |train_ser_loss: 1.3672 |train_accuracy: 0.2435 |test_ser_loss: 1.3792 |test_accuracy: 0.3124
test: 
 [[  0   0 170   0]
 [  0   0 435   0]
 [  0   0 384   0]
 [  0   0 240   0]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       170
          1       0.00      0.00      0.00       435
          2       0.31      1.00      0.48       384
          3       0.00      0.00      0.00       240

avg / total       0.10      0.31      0.15      1229

---testing ends---
Time:  0.744659218005836
-----epoch:  1 -----
--training begins--
--training ends--
--testing begins--
Epoch: 1 |train_ser_loss: 1.3574 |train_accuracy: 0.3091 |test_ser_loss: 1.3671 |test_accuracy: 0.3124
test: 
 [[  0   0 170   0]
 [  0   0 435   0]
 [  0   0 384   0]
 [  0   0 240   0]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       170
          1       0.00      0.00     

--training ends--
--testing begins--
Epoch: 13 |train_ser_loss: 1.1369 |train_accuracy: 0.4135 |test_ser_loss: 1.2163 |test_accuracy: 0.4117
test: 
 [[ 90  46  21  13]
 [ 90 207  74  64]
 [ 89 138  85  72]
 [ 27  24  65 124]]
             precision    recall  f1-score   support

          0       0.30      0.53      0.39       170
          1       0.50      0.48      0.49       435
          2       0.35      0.22      0.27       384
          3       0.45      0.52      0.48       240

avg / total       0.42      0.41      0.40      1229

---testing ends---
Time:  0.7910924004390836
-----epoch:  14 -----
--training begins--
--training ends--
--testing begins--
Epoch: 14 |train_ser_loss: 1.1525 |train_accuracy: 0.4125 |test_ser_loss: 1.2163 |test_accuracy: 0.4076
test: 
 [[ 94  45  19  12]
 [100 205  70  60]
 [ 96 138  79  71]
 [ 31  22  64 123]]
             precision    recall  f1-score   support

          0       0.29      0.55      0.38       170
          1       0.50      0.47 

--training ends--
--testing begins--
Epoch: 26 |train_ser_loss: 1.0998 |train_accuracy: 0.4348 |test_ser_loss: 1.1923 |test_accuracy: 0.4223
test: 
 [[ 90  47  20  13]
 [ 91 206  78  60]
 [ 84 142  83  75]
 [ 22  23  55 140]]
             precision    recall  f1-score   support

          0       0.31      0.53      0.39       170
          1       0.49      0.47      0.48       435
          2       0.35      0.22      0.27       384
          3       0.49      0.58      0.53       240

avg / total       0.42      0.42      0.41      1229

---testing ends---
Time:  0.6804108340293169
-----epoch:  27 -----
--training begins--
--training ends--
--testing begins--
Epoch: 27 |train_ser_loss: 1.1061 |train_accuracy: 0.4437 |test_ser_loss: 1.1929 |test_accuracy: 0.4239
test: 
 [[ 94  45  18  13]
 [ 94 206  74  61]
 [ 84 144  79  77]
 [ 27  24  47 142]]
             precision    recall  f1-score   support

          0       0.31      0.55      0.40       170
          1       0.49      0.47 

--training ends--
--testing begins--
Epoch: 39 |train_ser_loss: 1.0940 |train_accuracy: 0.4584 |test_ser_loss: 1.1819 |test_accuracy: 0.4280
test: 
 [[ 89  46  21  14]
 [ 82 202  95  56]
 [ 76 141  96  71]
 [ 29  21  51 139]]
             precision    recall  f1-score   support

          0       0.32      0.52      0.40       170
          1       0.49      0.46      0.48       435
          2       0.37      0.25      0.30       384
          3       0.50      0.58      0.53       240

avg / total       0.43      0.43      0.42      1229

---testing ends---
Time:  0.7365285661071539
-----epoch:  40 -----
--training begins--
--training ends--
--testing begins--
Epoch: 40 |train_ser_loss: 1.0872 |train_accuracy: 0.4538 |test_ser_loss: 1.1795 |test_accuracy: 0.4272
test: 
 [[ 82  48  26  14]
 [ 73 206  97  59]
 [ 71 143  94  76]
 [ 27  21  49 143]]
             precision    recall  f1-score   support

          0       0.32      0.48      0.39       170
          1       0.49      0.47 

--training ends--
--testing begins--
Epoch: 52 |train_ser_loss: 1.0966 |train_accuracy: 0.4701 |test_ser_loss: 1.1747 |test_accuracy: 0.4353
test: 
 [[ 90  41  26  13]
 [ 72 206  99  58]
 [ 73 137 100  74]
 [ 26  20  55 139]]
             precision    recall  f1-score   support

          0       0.34      0.53      0.42       170
          1       0.51      0.47      0.49       435
          2       0.36      0.26      0.30       384
          3       0.49      0.58      0.53       240

avg / total       0.44      0.44      0.43      1229

---testing ends---
Time:  0.9402661891654134
-----epoch:  53 -----
--training begins--
--training ends--
--testing begins--
Epoch: 53 |train_ser_loss: 1.0644 |train_accuracy: 0.4704 |test_ser_loss: 1.1716 |test_accuracy: 0.4418
test: 
 [[ 88  43  25  14]
 [ 64 211 102  58]
 [ 67 139 103  75]
 [ 24  20  55 141]]
             precision    recall  f1-score   support

          0       0.36      0.52      0.43       170
          1       0.51      0.49 

--training ends--
--testing begins--
Epoch: 65 |train_ser_loss: 1.0382 |train_accuracy: 0.4849 |test_ser_loss: 1.1680 |test_accuracy: 0.4483
test: 
 [[ 95  38  26  11]
 [ 68 203 108  56]
 [ 68 131 112  73]
 [ 25  20  54 141]]
             precision    recall  f1-score   support

          0       0.37      0.56      0.45       170
          1       0.52      0.47      0.49       435
          2       0.37      0.29      0.33       384
          3       0.50      0.59      0.54       240

avg / total       0.45      0.45      0.44      1229

---testing ends---
Time:  0.7029267773032188
-----epoch:  66 -----
--training begins--
--training ends--
--testing begins--
Epoch: 66 |train_ser_loss: 1.0201 |train_accuracy: 0.4767 |test_ser_loss: 1.1669 |test_accuracy: 0.4410
test: 
 [[ 93  40  26  11]
 [ 66 195 119  55]
 [ 66 130 113  75]
 [ 23  20  56 141]]
             precision    recall  f1-score   support

          0       0.38      0.55      0.44       170
          1       0.51      0.45 

--training ends--
--testing begins--
Epoch: 78 |train_ser_loss: 1.0215 |train_accuracy: 0.4912 |test_ser_loss: 1.1626 |test_accuracy: 0.4516
test: 
 [[ 92  41  26  11]
 [ 62 192 126  55]
 [ 66 122 130  66]
 [ 22  19  58 141]]
             precision    recall  f1-score   support

          0       0.38      0.54      0.45       170
          1       0.51      0.44      0.47       435
          2       0.38      0.34      0.36       384
          3       0.52      0.59      0.55       240

avg / total       0.45      0.45      0.45      1229

---testing ends---
Time:  0.8621515035629272
-----epoch:  79 -----
--training begins--
--training ends--
--testing begins--
Epoch: 79 |train_ser_loss: 0.9908 |train_accuracy: 0.4936 |test_ser_loss: 1.1641 |test_accuracy: 0.4508
test: 
 [[ 94  40  26  10]
 [ 66 196 118  55]
 [ 70 123 126  65]
 [ 25  19  58 138]]
             precision    recall  f1-score   support

          0       0.37      0.55      0.44       170
          1       0.52      0.45 

--training ends--
--testing begins--
Epoch: 91 |train_ser_loss: 0.9837 |train_accuracy: 0.4978 |test_ser_loss: 1.1568 |test_accuracy: 0.4516
test: 
 [[ 94  37  29  10]
 [ 61 194 125  55]
 [ 65 121 122  76]
 [ 18  20  57 145]]
             precision    recall  f1-score   support

          0       0.39      0.55      0.46       170
          1       0.52      0.45      0.48       435
          2       0.37      0.32      0.34       384
          3       0.51      0.60      0.55       240

avg / total       0.45      0.45      0.45      1229

---testing ends---
Time:  0.7393288565799594
-----epoch:  92 -----
--training begins--
--training ends--
--testing begins--
Epoch: 92 |train_ser_loss: 0.9976 |train_accuracy: 0.5015 |test_ser_loss: 1.1514 |test_accuracy: 0.4573
test: 
 [[ 88  38  34  10]
 [ 55 200 126  54]
 [ 61 121 130  72]
 [ 17  21  58 144]]
             precision    recall  f1-score   support

          0       0.40      0.52      0.45       170
          1       0.53      0.46 

In [ ]:
session 1

--layer 0--
Epoch: 97 |train_ser_loss: 0.3623 |train_accuracy: 0.6460 |test_ser_loss: 1.0066 |test_accuracy: 0.5503
test: 
 [[148  44  26  11]
 [ 34  99 111  26]
 [  5  44 176 157]
 [  4   2  19 168]]
             precision    recall  f1-score   support

          0       0.77      0.65      0.70       229
          1       0.52      0.37      0.43       270
          2       0.53      0.46      0.49       382
          3       0.46      0.87      0.61       193

avg / total       0.57      0.55      0.54      1074


--layer 1--
Epoch: 99 |train_ser_loss: 0.1650 |train_accuracy: 0.6814 |test_ser_loss: 0.9590 |test_accuracy: 0.6043
test: 
 [[150  43  23  13]
 [ 20 151  70  29]
 [ 10  48 179 145]
 [  4   2  18 169]]
             precision    recall  f1-score   support

          0       0.82      0.66      0.73       229
          1       0.62      0.56      0.59       270
          2       0.62      0.47      0.53       382
          3       0.47      0.88      0.62       193

avg / total       0.63      0.60      0.60      1074

--layer 2--
Epoch: 97 |train_ser_loss: 0.0750 |train_accuracy: 0.7110 |test_ser_loss: 0.9655 |test_accuracy: 0.6229
test: 
 [[152  37  31   9]
 [ 14 152  72  32]
 [ 11  36 193 142]
 [  5   1  15 172]]
             precision    recall  f1-score   support

          0       0.84      0.66      0.74       229
          1       0.67      0.56      0.61       270
          2       0.62      0.51      0.56       382
          3       0.48      0.89      0.63       193

avg / total       0.65      0.62      0.62      1074

--layer 3--
Epoch: 42 |train_ser_loss: 0.1702 |train_accuracy: 0.6943 |test_ser_loss: 0.9268 |test_accuracy: 0.6266
test: 
 [[152  32  30  15]
 [  9 162  59  40]
 [ 12  30 180 160]
 [  5   3   6 179]]
             precision    recall  f1-score   support

          0       0.85      0.66      0.75       229
          1       0.71      0.60      0.65       270
          2       0.65      0.47      0.55       382
          3       0.45      0.93      0.61       193

avg / total       0.68      0.63      0.63      1074

--layer 4--
Epoch: 93 |train_ser_loss: 0.1202 |train_accuracy: 0.7085 |test_ser_loss: 0.9271 |test_accuracy: 0.6378
test: 
 [[146  44  27  12]
 [ 11 155  76  28]
 [  7  39 214 122]
 [  2   2  19 170]]
             precision    recall  f1-score   support

          0       0.88      0.64      0.74       229
          1       0.65      0.57      0.61       270
          2       0.64      0.56      0.60       382
          3       0.51      0.88      0.65       193

avg / total       0.67      0.64      0.64      1074


--layer 5--
Epoch: 90 |train_ser_loss: 0.0712 |train_accuracy: 0.7404 |test_ser_loss: 0.8727 |test_accuracy: 0.6601
test: 
 [[139  46  35   9]
 [  6 171  63  30]
 [  7  28 221 126]
 [  1   4  10 178]]
             precision    recall  f1-score   support

          0       0.91      0.61      0.73       229
          1       0.69      0.63      0.66       270
          2       0.67      0.58      0.62       382
          3       0.52      0.92      0.66       193

avg / total       0.70      0.66      0.66      1074


--layer 6--
Epoch: 63 |train_ser_loss: 0.0795 |train_accuracy: 0.7427 |test_ser_loss: 0.8658 |test_accuracy: 0.6667
test: 
 [[156  26  37  10]
 [  7 182  51  30]
 [  8  35 205 134]
 [  2   4  14 173]]
             precision    recall  f1-score   support

          0       0.90      0.68      0.78       229
          1       0.74      0.67      0.70       270
          2       0.67      0.54      0.60       382
          3       0.50      0.90      0.64       193

avg / total       0.70      0.67      0.67      1074


--layer 7--
Epoch: 99 |train_ser_loss: 0.0466 |train_accuracy: 0.7501 |test_ser_loss: 0.9238 |test_accuracy: 0.6601
test: 
 [[154  31  36   8]
 [  5 175  62  28]
 [ 14  19 211 138]
 [  1   2  21 169]]
             precision    recall  f1-score   support

          0       0.89      0.67      0.76       229
          1       0.77      0.65      0.70       270
          2       0.64      0.55      0.59       382
          3       0.49      0.88      0.63       193

avg / total       0.70      0.66      0.66      1074

--layer 8--
Epoch: 78 |train_ser_loss: 0.1067 |train_accuracy: 0.7528 |test_ser_loss: 0.8848 |test_accuracy: 0.6657
test: 
 [[164  27  31   7]
 [  8 187  43  32]
 [ 14  41 196 131]
 [  2   7  16 168]]
             precision    recall  f1-score   support

          0       0.87      0.72      0.79       229
          1       0.71      0.69      0.70       270
          2       0.69      0.51      0.59       382
          3       0.50      0.87      0.63       193

avg / total       0.70      0.67      0.67      1074

--layer 9--
Epoch: 97 |train_ser_loss: 0.1680 |train_accuracy: 0.7248 |test_ser_loss: 0.8691 |test_accuracy: 0.6499
test: 
 [[165  25  29  10]
 [  8 165  63  34]
 [ 15  31 204 132]
 [  4   3  22 164]]
             precision    recall  f1-score   support

          0       0.86      0.72      0.78       229
          1       0.74      0.61      0.67       270
          2       0.64      0.53      0.58       382
          3       0.48      0.85      0.62       193

avg / total       0.68      0.65      0.65      1074

--layer 10--
Epoch: 96 |train_ser_loss: 0.2384 |train_accuracy: 0.6229 |test_ser_loss: 0.9488 |test_accuracy: 0.5857
test: 
 [[159  31  31   8]
 [ 19 134  84  33]
 [ 11  51 170 150]
 [  3   0  24 166]]
             precision    recall  f1-score   support

          0       0.83      0.69      0.76       229
          1       0.62      0.50      0.55       270
          2       0.55      0.45      0.49       382
          3       0.46      0.86      0.60       193

avg / total       0.61      0.59      0.58      1074

--layer 11--
Epoch: 98 |train_ser_loss: 0.5452 |train_accuracy: 0.5079 |test_ser_loss: 1.0761 |test_accuracy: 0.5279
test: 
 [[146  30  36  17]
 [ 26 162  45  37]
 [ 36  77 101 168]
 [  4   8  23 158]]
             precision    recall  f1-score   support

          0       0.69      0.64      0.66       229
          1       0.58      0.60      0.59       270
          2       0.49      0.26      0.34       382
          3       0.42      0.82      0.55       193

avg / total       0.54      0.53      0.51      1074

--layer 12--
Epoch: 81 |train_ser_loss: 0.7205 |train_accuracy: 0.4577 |test_ser_loss: 1.1474 |test_accuracy: 0.4655
test: 
 [[118  38  53  20]
 [ 24 168  28  50]
 [ 40 100  59 183]
 [  5  19  14 155]]
             precision    recall  f1-score   support

          0       0.63      0.52      0.57       229
          1       0.52      0.62      0.56       270
          2       0.38      0.15      0.22       382
          3       0.38      0.80      0.52       193

avg / total       0.47      0.47      0.43      1074

In [ ]:
session 2

--layer 0--
Epoch: 59 |train_ser_loss: 0.8226 |train_accuracy: 0.6088 |test_ser_loss: 0.9236 |test_accuracy: 0.6043
test: 
 [[ 88  37  11   1]
 [ 25 179 105  16]
 [ 11  76 242  33]
 [  6   3  80 108]]
             precision    recall  f1-score   support

          0       0.68      0.64      0.66       137
          1       0.61      0.55      0.58       325
          2       0.55      0.67      0.60       362
          3       0.68      0.55      0.61       197

avg / total       0.61      0.60      0.60      1021


--layer 1--
Epoch: 99 |train_ser_loss: 0.6695 |train_accuracy: 0.7028 |test_ser_loss: 0.8498 |test_accuracy: 0.6494
test: 
 [[ 96  25  16   0]
 [ 25 219  71  10]
 [ 10  98 227  27]
 [  3  13  60 121]]
             precision    recall  f1-score   support

          0       0.72      0.70      0.71       137
          1       0.62      0.67      0.64       325
          2       0.61      0.63      0.62       362
          3       0.77      0.61      0.68       197

avg / total       0.66      0.65      0.65      1021

--layer 2--
Epoch: 95 |train_ser_loss: 0.5665 |train_accuracy: 0.7240 |test_ser_loss: 0.8567 |test_accuracy: 0.6327
test: 
 [[103  22  12   0]
 [ 32 200  82  11]
 [ 16  99 225  22]
 [  5  15  59 118]]
             precision    recall  f1-score   support

          0       0.66      0.75      0.70       137
          1       0.60      0.62      0.61       325
          2       0.60      0.62      0.61       362
          3       0.78      0.60      0.68       197

avg / total       0.64      0.63      0.63      1021


--layer 3--
Epoch: 78 |train_ser_loss: 0.6254 |train_accuracy: 0.7285 |test_ser_loss: 0.8485 |test_accuracy: 0.6445
test: 
 [[ 87  32  18   0]
 [ 17 207  87  14]
 [  7  91 242  22]
 [  1  17  57 122]]
             precision    recall  f1-score   support

          0       0.78      0.64      0.70       137
          1       0.60      0.64      0.62       325
          2       0.60      0.67      0.63       362
          3       0.77      0.62      0.69       197

avg / total       0.66      0.64      0.65      1021

--layer 4--
Epoch: 98 |train_ser_loss: 0.5779 |train_accuracy: 0.7417 |test_ser_loss: 0.8653 |test_accuracy: 0.6621
test: 
 [[ 93  33  10   1]
 [ 17 239  59  10]
 [  7 105 225  25]
 [  2  15  61 119]]
             precision    recall  f1-score   support

          0       0.78      0.68      0.73       137
          1       0.61      0.74      0.67       325
          2       0.63      0.62      0.63       362
          3       0.77      0.60      0.68       197

avg / total       0.67      0.66      0.66      1021

--layer 5--
Epoch: 98 |train_ser_loss: 0.6393 |train_accuracy: 0.7205 |test_ser_loss: 0.9000 |test_accuracy: 0.6327
test: 
 [[103  26   8   0]
 [ 22 208  90   5]
 [  9 101 231  21]
 [  1   9  83 104]]
             precision    recall  f1-score   support

          0       0.76      0.75      0.76       137
          1       0.60      0.64      0.62       325
          2       0.56      0.64      0.60       362
          3       0.80      0.53      0.64       197

avg / total       0.65      0.63      0.63      1021

--layer 6--
Epoch: 84 |train_ser_loss: 0.6556 |train_accuracy: 0.7098 |test_ser_loss: 0.8398 |test_accuracy: 0.6592
test: 
 [[111  21   4   1]
 [ 26 224  66   9]
 [ 11 102 228  21]
 [  3   3  81 110]]
             precision    recall  f1-score   support

          0       0.74      0.81      0.77       137
          1       0.64      0.69      0.66       325
          2       0.60      0.63      0.62       362
          3       0.78      0.56      0.65       197

avg / total       0.67      0.66      0.66      1021

--layer 7--
Epoch: 65 |train_ser_loss: 0.6386 |train_accuracy: 0.7305 |test_ser_loss: 0.8309 |test_accuracy: 0.6709
test: 
 [[110  18   8   1]
 [ 27 233  48  17]
 [ 21 102 214  25]
 [  3  22  44 128]]
             precision    recall  f1-score   support

          0       0.68      0.80      0.74       137
          1       0.62      0.72      0.67       325
          2       0.68      0.59      0.63       362
          3       0.75      0.65      0.70       197

avg / total       0.68      0.67      0.67      1021

--layer 8--
Epoch: 99 |train_ser_loss: 0.6520 |train_accuracy: 0.7484 |test_ser_loss: 0.9087 |test_accuracy: 0.6366
test: 
 [[110  15  11   1]
 [ 25 230  60  10]
 [ 21 112 201  28]
 [  4  21  63 109]]
             precision    recall  f1-score   support

          0       0.69      0.80      0.74       137
          1       0.61      0.71      0.65       325
          2       0.60      0.56      0.58       362
          3       0.74      0.55      0.63       197

avg / total       0.64      0.64      0.63      1021

--layer 9--
Epoch: 77 |train_ser_loss: 0.6811 |train_accuracy: 0.7127 |test_ser_loss: 0.8886 |test_accuracy: 0.6259
test: 
 [[105  23   9   0]
 [ 20 244  47  14]
 [ 32 129 181  20]
 [  5  36  47 109]]
             precision    recall  f1-score   support

          0       0.65      0.77      0.70       137
          1       0.56      0.75      0.64       325
          2       0.64      0.50      0.56       362
          3       0.76      0.55      0.64       197

avg / total       0.64      0.63      0.62      1021

--layer 10--
Epoch: 98 |train_ser_loss: 0.8573 |train_accuracy: 0.6446 |test_ser_loss: 0.9683 |test_accuracy: 0.5661
test: 
 [[105  24   6   2]
 [ 33 183 100   9]
 [ 25 133 189  15]
 [ 10   4  82 101]]
             precision    recall  f1-score   support

          0       0.61      0.77      0.68       137
          1       0.53      0.56      0.55       325
          2       0.50      0.52      0.51       362
          3       0.80      0.51      0.62       197

avg / total       0.58      0.57      0.57      1021

--layer 11--
Epoch: 88 |train_ser_loss: 1.1568 |train_accuracy: 0.4816 |test_ser_loss: 1.1461 |test_accuracy: 0.4417
test: 
 [[101  29   6   1]
 [ 66 189  63   7]
 [104 155  76  27]
 [ 29  21  62  85]]
             precision    recall  f1-score   support

          0       0.34      0.74      0.46       137
          1       0.48      0.58      0.53       325
          2       0.37      0.21      0.27       362
          3       0.71      0.43      0.54       197

avg / total       0.46      0.44      0.43      1021

--layer 12--
Epoch: 99 |train_ser_loss: 1.1301 |train_accuracy: 0.4914 |test_ser_loss: 1.1533 |test_accuracy: 0.4319
test: 
 [[ 83  43   4   7]
 [ 52 221  26  26]
 [ 81 192  43  46]
 [ 28  29  46  94]]
             precision    recall  f1-score   support

          0       0.34      0.61      0.44       137
          1       0.46      0.68      0.55       325
          2       0.36      0.12      0.18       362
          3       0.54      0.48      0.51       197

avg / total       0.42      0.43      0.39      1021

In [ ]:
session 3

--layer 0--
Epoch: 99 |train_ser_loss: 0.7102 |train_accuracy: 0.6515 |test_ser_loss: 1.0781 |test_accuracy: 0.5391
test: 
 [[132  58  39  11]
 [ 29  83  91  83]
 [  9  62 154  94]
 [  1   3  50 251]]
             precision    recall  f1-score   support

          0       0.77      0.55      0.64       240
          1       0.40      0.29      0.34       286
          2       0.46      0.48      0.47       319
          3       0.57      0.82      0.67       305

avg / total       0.54      0.54      0.53      1150


--layer 1--
Epoch: 98 |train_ser_loss: 0.7233 |train_accuracy: 0.6775 |test_ser_loss: 1.0509 |test_accuracy: 0.5783
test: 
 [[144  46  38  12]
 [ 26  95 105  60]
 [ 17  49 191  62]
 [  3   6  61 235]]
             precision    recall  f1-score   support

          0       0.76      0.60      0.67       240
          1       0.48      0.33      0.39       286
          2       0.48      0.60      0.54       319
          3       0.64      0.77      0.70       305

avg / total       0.58      0.58      0.57      1150

--layer 2--
Epoch: 95 |train_ser_loss: 0.6407 |train_accuracy: 0.7156 |test_ser_loss: 1.0214 |test_accuracy: 0.6035
test: 
 [[156  33  35  16]
 [ 28  93 102  63]
 [ 21  23 224  51]
 [  5   5  74 221]]
             precision    recall  f1-score   support

          0       0.74      0.65      0.69       240
          1       0.60      0.33      0.42       286
          2       0.51      0.70      0.59       319
          3       0.63      0.72      0.67       305

avg / total       0.62      0.60      0.59      1150



--layer 3--
Epoch: 99 |train_ser_loss: 0.5950 |train_accuracy: 0.7301 |test_ser_loss: 1.0324 |test_accuracy: 0.6052
test: 
 [[158  32  36  14]
 [ 29 101  82  74]
 [ 26  35 203  55]
 [  4   5  62 234]]
             precision    recall  f1-score   support

          0       0.73      0.66      0.69       240
          1       0.58      0.35      0.44       286
          2       0.53      0.64      0.58       319
          3       0.62      0.77      0.69       305

avg / total       0.61      0.61      0.60      1150

--layer 4--
Epoch: 92 |train_ser_loss: 0.5702 |train_accuracy: 0.7499 |test_ser_loss: 0.9714 |test_accuracy: 0.6226
test: 
 [[162  27  39  12]
 [ 31 104  72  79]
 [ 23  19 212  65]
 [  5   3  59 238]]
             precision    recall  f1-score   support

          0       0.73      0.68      0.70       240
          1       0.68      0.36      0.47       286
          2       0.55      0.66      0.60       319
          3       0.60      0.78      0.68       305

avg / total       0.64      0.62      0.61      1150


--layer 5--
Epoch: 65 |train_ser_loss: 0.6071 |train_accuracy: 0.7400 |test_ser_loss: 0.9479 |test_accuracy: 0.6383
test: 
 [[171  25  37   7]
 [ 31 124  68  63]
 [ 34  28 199  58]
 [  3   8  54 240]]
             precision    recall  f1-score   support

          0       0.72      0.71      0.71       240
          1       0.67      0.43      0.53       286
          2       0.56      0.62      0.59       319
          3       0.65      0.79      0.71       305

avg / total       0.64      0.64      0.63      1150


--layer 6--
Epoch: 92 |train_ser_loss: 0.5583 |train_accuracy: 0.7674 |test_ser_loss: 0.9587 |test_accuracy: 0.6470
test: 
 [[170  26  36   8]
 [ 31 118  69  68]
 [ 30  28 211  50]
 [  6   7  47 245]]
             precision    recall  f1-score   support

          0       0.72      0.71      0.71       240
          1       0.66      0.41      0.51       286
          2       0.58      0.66      0.62       319
          3       0.66      0.80      0.72       305

avg / total       0.65      0.65      0.64      1150

--layer 7--
Epoch: 93 |train_ser_loss: 0.5306 |train_accuracy: 0.7738 |test_ser_loss: 0.9500 |test_accuracy: 0.6452
test: 
 [[161  30  37  12]
 [ 18 129  68  71]
 [ 27  34 202  56]
 [  3   8  44 250]]
             precision    recall  f1-score   support

          0       0.77      0.67      0.72       240
          1       0.64      0.45      0.53       286
          2       0.58      0.63      0.60       319
          3       0.64      0.82      0.72       305

avg / total       0.65      0.65      0.64      1150


--layer 8--
Epoch: 99 |train_ser_loss: 0.5837 |train_accuracy: 0.7598 |test_ser_loss: 0.9822 |test_accuracy: 0.6330
test: 
 [[156  27  43  14]
 [ 17 115  81  73]
 [ 23  28 210  58]
 [  5   7  46 247]]
             precision    recall  f1-score   support

          0       0.78      0.65      0.71       240
          1       0.65      0.40      0.50       286
          2       0.55      0.66      0.60       319
          3       0.63      0.81      0.71       305

avg / total       0.64      0.63      0.63      1150


--layer 9--
Epoch: 98 |train_ser_loss: 0.6380 |train_accuracy: 0.7366 |test_ser_loss: 0.9978 |test_accuracy: 0.6183
test: 
 [[151  26  52  11]
 [ 17 119  80  70]
 [ 27  27 209  56]
 [  6   8  59 232]]
             precision    recall  f1-score   support

          0       0.75      0.63      0.68       240
          1       0.66      0.42      0.51       286
          2       0.52      0.66      0.58       319
          3       0.63      0.76      0.69       305

avg / total       0.63      0.62      0.61      1150


--layer 10--
Epoch: 99 |train_ser_loss: 0.6975 |train_accuracy: 0.6570 |test_ser_loss: 1.0741 |test_accuracy: 0.5643
test: 
 [[122  44  62  12]
 [ 14 102  95  75]
 [  8  40 182  89]
 [ 11   6  45 243]]
             precision    recall  f1-score   support

          0       0.79      0.51      0.62       240
          1       0.53      0.36      0.43       286
          2       0.47      0.57      0.52       319
          3       0.58      0.80      0.67       305

avg / total       0.58      0.56      0.56      1150

--layer 11--
Epoch: 85 |train_ser_loss: 1.0383 |train_accuracy: 0.4961 |test_ser_loss: 1.2377 |test_accuracy: 0.4357
test: 
 [[101  63  64  12]
 [ 30  85 103  68]
 [ 29  84 113  93]
 [ 20  17  66 202]]
             precision    recall  f1-score   support

          0       0.56      0.42      0.48       240
          1       0.34      0.30      0.32       286
          2       0.33      0.35      0.34       319
          3       0.54      0.66      0.59       305

avg / total       0.44      0.44      0.43      1150

--layer 12--
Epoch: 91 |train_ser_loss: 1.0879 |train_accuracy: 0.4860 |test_ser_loss: 1.2559 |test_accuracy: 0.4191
test: 
 [[ 93  73  51  23]
 [ 22 110  90  64]
 [ 29 102 100  88]
 [ 13  16  97 179]]
             precision    recall  f1-score   support

          0       0.59      0.39      0.47       240
          1       0.37      0.38      0.37       286
          2       0.30      0.31      0.30       319
          3       0.51      0.59      0.54       305

avg / total       0.43      0.42      0.42      1150

In [ ]:
session 4

--layer 0--
Epoch: 98 |train_ser_loss: 0.7782 |train_accuracy: 0.6759 |test_ser_loss: 0.9745 |test_accuracy: 0.5799
test: 
 [[257  40  28   2]
 [ 97 133  55  14]
 [ 27  76 140  14]
 [  7   2  69  65]]
             precision    recall  f1-score   support

          0       0.66      0.79      0.72       327
          1       0.53      0.44      0.48       299
          2       0.48      0.54      0.51       257
          3       0.68      0.45      0.55       143

avg / total       0.58      0.58      0.57      1026


--layer 1--
Epoch: 97 |train_ser_loss: 0.8025 |train_accuracy: 0.6802 |test_ser_loss: 1.0254 |test_accuracy: 0.5731
test: 
 [[267  41  15   4]
 [114 126  45  14]
 [ 33  90 124  10]
 [ 10   4  58  71]]
             precision    recall  f1-score   support

          0       0.63      0.82      0.71       327
          1       0.48      0.42      0.45       299
          2       0.51      0.48      0.50       257
          3       0.72      0.50      0.59       143

avg / total       0.57      0.57      0.56      1026


--layer 2--
Epoch: 93 |train_ser_loss: 0.7215 |train_accuracy: 0.7409 |test_ser_loss: 0.9641 |test_accuracy: 0.6277
test: 
 [[285  17  24   1]
 [112 134  39  14]
 [ 30  71 147   9]
 [ 11   6  48  78]]
             precision    recall  f1-score   support

          0       0.65      0.87      0.75       327
          1       0.59      0.45      0.51       299
          2       0.57      0.57      0.57       257
          3       0.76      0.55      0.64       143

avg / total       0.63      0.63      0.62      1026




--layer 3--
Epoch: 81 |train_ser_loss: 0.7496 |train_accuracy: 0.7333 |test_ser_loss: 0.9873 |test_accuracy: 0.6345
test: 
 [[282  16  26   3]
 [108 119  53  19]
 [ 34  51 163   9]
 [  7   2  47  87]]
             precision    recall  f1-score   support

          0       0.65      0.86      0.74       327
          1       0.63      0.40      0.49       299
          2       0.56      0.63      0.60       257
          3       0.74      0.61      0.67       143

avg / total       0.64      0.63      0.62      1026


--layer 4--
Epoch: 79 |train_ser_loss: 0.7948 |train_accuracy: 0.7117 |test_ser_loss: 0.9959 |test_accuracy: 0.6384
test: 
 [[277  28  16   6]
 [ 91 143  52  13]
 [ 32  69 144  12]
 [ 10   2  40  91]]
             precision    recall  f1-score   support

          0       0.68      0.85      0.75       327
          1       0.59      0.48      0.53       299
          2       0.57      0.56      0.57       257
          3       0.75      0.64      0.69       143

avg / total       0.63      0.64      0.63      1026


--layer 5--
Epoch: 94 |train_ser_loss: 0.5626 |train_accuracy: 0.7633 |test_ser_loss: 0.9955 |test_accuracy: 0.6384
test: 
 [[287   7  29   4]
 [105 122  63   9]
 [ 41  42 161  13]
 [  6   2  50  85]]
             precision    recall  f1-score   support

          0       0.65      0.88      0.75       327
          1       0.71      0.41      0.52       299
          2       0.53      0.63      0.58       257
          3       0.77      0.59      0.67       143

avg / total       0.65      0.64      0.63      1026


--layer 6--
Epoch: 98 |train_ser_loss: 0.6487 |train_accuracy: 0.7593 |test_ser_loss: 0.9552 |test_accuracy: 0.6316
test: 
 [[268  32  21   6]
 [ 89 140  58  12]
 [ 33  64 147  13]
 [  5   2  43  93]]
             precision    recall  f1-score   support

          0       0.68      0.82      0.74       327
          1       0.59      0.47      0.52       299
          2       0.55      0.57      0.56       257
          3       0.75      0.65      0.70       143

avg / total       0.63      0.63      0.63      1026

--layer 7--
Epoch: 99 |train_ser_loss: 0.6952 |train_accuracy: 0.7338 |test_ser_loss: 0.9639 |test_accuracy: 0.6404
test: 
 [[280  10  37   0]
 [ 75 136  77  11]
 [ 40  56 151  10]
 [  6   0  47  90]]
             precision    recall  f1-score   support

          0       0.70      0.86      0.77       327
          1       0.67      0.45      0.54       299
          2       0.48      0.59      0.53       257
          3       0.81      0.63      0.71       143

avg / total       0.65      0.64      0.64      1026


--layer 8--
Epoch: 97 |train_ser_loss: 0.6606 |train_accuracy: 0.7483 |test_ser_loss: 0.9484 |test_accuracy: 0.6491
test: 
 [[265  31  28   3]
 [ 63 171  51  14]
 [ 34  68 141  14]
 [  5   7  42  89]]
             precision    recall  f1-score   support

          0       0.72      0.81      0.76       327
          1       0.62      0.57      0.59       299
          2       0.54      0.55      0.54       257
          3       0.74      0.62      0.68       143

avg / total       0.65      0.65      0.65      1026


--layer 9--
Epoch: 99 |train_ser_loss: 0.6898 |train_accuracy: 0.7461 |test_ser_loss: 0.9300 |test_accuracy: 0.6384
test: 
 [[282  17  26   2]
 [ 83 144  62  10]
 [ 45  58 143  11]
 [  9  11  37  86]]
             precision    recall  f1-score   support

          0       0.67      0.86      0.76       327
          1       0.63      0.48      0.54       299
          2       0.53      0.56      0.54       257
          3       0.79      0.60      0.68       143

avg / total       0.64      0.64      0.63      1026


--layer 10--
Epoch: 83 |train_ser_loss: 0.8138 |train_accuracy: 0.6191 |test_ser_loss: 0.9324 |test_accuracy: 0.5828
test: 
 [[251  50  17   9]
 [ 77 157  52  13]
 [ 30 102 110  15]
 [  9   4  50  80]]
             precision    recall  f1-score   support

          0       0.68      0.77      0.72       327
          1       0.50      0.53      0.51       299
          2       0.48      0.43      0.45       257
          3       0.68      0.56      0.62       143

avg / total       0.58      0.58      0.58      1026


--layer 11--
Epoch: 99 |train_ser_loss: 1.0650 |train_accuracy: 0.4897 |test_ser_loss: 1.1037 |test_accuracy: 0.5136
test: 
 [[187  52  63  25]
 [ 47 137  83  32]
 [ 36  87  92  42]
 [ 10   4  18 111]]
             precision    recall  f1-score   support

          0       0.67      0.57      0.62       327
          1       0.49      0.46      0.47       299
          2       0.36      0.36      0.36       257
          3       0.53      0.78      0.63       143

avg / total       0.52      0.51      0.51      1026

--layer 12--
Epoch: 79 |train_ser_loss: 1.0476 |train_accuracy: 0.4736 |test_ser_loss: 1.1388 |test_accuracy: 0.4903
test: 
 [[211  54  43  19]
 [ 70 145  62  22]
 [ 59 108  57  33]
 [ 26   5  22  90]]
             precision    recall  f1-score   support

          0       0.58      0.65      0.61       327
          1       0.46      0.48      0.47       299
          2       0.31      0.22      0.26       257
          3       0.55      0.63      0.59       143

avg / total       0.47      0.49      0.48      1026

In [ ]:
session 5

--layer 0--
Epoch: 98 |train_ser_loss: 0.9592 |train_accuracy: 0.6631 |test_ser_loss: 1.0154 |test_accuracy: 0.5557
test: 
 [[129  21  15   5]
 [ 86 144 163  42]
 [ 25  36 280  43]
 [ 16  12  82 130]]
             precision    recall  f1-score   support

          0       0.50      0.76      0.61       170
          1       0.68      0.33      0.44       435
          2       0.52      0.73      0.61       384
          3       0.59      0.54      0.57       240

avg / total       0.59      0.56      0.54      1229


--layer 1--
Epoch: 98 |train_ser_loss: 0.8785 |train_accuracy: 0.6816 |test_ser_loss: 1.0055 |test_accuracy: 0.5663
test: 
 [[128  30   8   4]
 [ 74 192 135  34]
 [ 23  80 241  40]
 [ 13  15  77 135]]
             precision    recall  f1-score   support

          0       0.54      0.75      0.63       170
          1       0.61      0.44      0.51       435
          2       0.52      0.63      0.57       384
          3       0.63      0.56      0.60       240

avg / total       0.58      0.57      0.56      1229


--layer 2--
Epoch: 99 |train_ser_loss: 0.7029 |train_accuracy: 0.7151 |test_ser_loss: 1.0149 |test_accuracy: 0.5842
test: 
 [[129  23  16   2]
 [ 45 208 154  28]
 [ 20  77 261  26]
 [  4  17  99 120]]
             precision    recall  f1-score   support

          0       0.65      0.76      0.70       170
          1       0.64      0.48      0.55       435
          2       0.49      0.68      0.57       384
          3       0.68      0.50      0.58       240

avg / total       0.60      0.58      0.58      1229




--layer 3--
poch: 96 |train_ser_loss: 0.7008 |train_accuracy: 0.7076 |test_ser_loss: 1.0032 |test_accuracy: 0.5810
test: 
 [[127  25  16   2]
 [ 74 195 141  25]
 [ 17  70 267  30]
 [ 15  15  85 125]]
             precision    recall  f1-score   support

          0       0.55      0.75      0.63       170
          1       0.64      0.45      0.53       435
          2       0.52      0.70      0.60       384
          3       0.69      0.52      0.59       240

avg / total       0.60      0.58      0.58      1229


--layer 4--
Epoch: 99 |train_ser_loss: 0.6890 |train_accuracy: 0.7616 |test_ser_loss: 0.8669 |test_accuracy: 0.6542
test: 
 [[133  21  14   2]
 [ 33 255 117  30]
 [ 19  43 294  28]
 [  9  50  59 122]]
             precision    recall  f1-score   support

          0       0.69      0.78      0.73       170
          1       0.69      0.59      0.63       435
          2       0.61      0.77      0.68       384
          3       0.67      0.51      0.58       240

avg / total       0.66      0.65      0.65      1229


--layer 5--
Epoch: 99 |train_ser_loss: 0.6518 |train_accuracy: 0.7647 |test_ser_loss: 0.9147 |test_accuracy: 0.6355
test: 
 [[110  30  29   1]
 [ 36 233 129  37]
 [  8  41 304  31]
 [  8  35  63 134]]
             precision    recall  f1-score   support

          0       0.68      0.65      0.66       170
          1       0.69      0.54      0.60       435
          2       0.58      0.79      0.67       384
          3       0.66      0.56      0.60       240

avg / total       0.65      0.64      0.63      1229


--layer 6--
Epoch: 95 |train_ser_loss: 0.5329 |train_accuracy: 0.7624 |test_ser_loss: 0.9253 |test_accuracy: 0.6566
test: 
 [[111  25  30   4]
 [ 23 249 134  29]
 [ 10  44 298  32]
 [  5  33  53 149]]
             precision    recall  f1-score   support

          0       0.74      0.65      0.70       170
          1       0.71      0.57      0.63       435
          2       0.58      0.78      0.66       384
          3       0.70      0.62      0.66       240

avg / total       0.67      0.66      0.66      1229

--layer 7--
Epoch: 88 |train_ser_loss: 0.4389 |train_accuracy: 0.7717 |test_ser_loss: 0.9244 |test_accuracy: 0.6461
test: 
 [[112  24  27   7]
 [ 31 245 129  30]
 [ 11  49 289  35]
 [  5  33  54 148]]
             precision    recall  f1-score   support

          0       0.70      0.66      0.68       170
          1       0.70      0.56      0.62       435
          2       0.58      0.75      0.65       384
          3       0.67      0.62      0.64       240

avg / total       0.66      0.65      0.65      1229


--layer 8--
Epoch: 95 |train_ser_loss: 0.5036 |train_accuracy: 0.7537 |test_ser_loss: 0.9919 |test_accuracy: 0.6225
test: 
 [[118  25  22   5]
 [ 27 236 144  28]
 [ 17  55 277  35]
 [  5  24  77 134]]
             precision    recall  f1-score   support

          0       0.71      0.69      0.70       170
          1       0.69      0.54      0.61       435
          2       0.53      0.72      0.61       384
          3       0.66      0.56      0.61       240

avg / total       0.64      0.62      0.62      1229


--layer 9--
Epoch: 91 |train_ser_loss: 0.5399 |train_accuracy: 0.7099 |test_ser_loss: 0.9357 |test_accuracy: 0.6208
test: 
 [[115  23  29   3]
 [ 36 223 146  30]
 [ 15  52 282  35]
 [  8  14  75 143]]
             precision    recall  f1-score   support

          0       0.66      0.68      0.67       170
          1       0.71      0.51      0.60       435
          2       0.53      0.73      0.62       384
          3       0.68      0.60      0.63       240

avg / total       0.64      0.62      0.62      1229


--layer 10--
Epoch: 99 |train_ser_loss: 0.7345 |train_accuracy: 0.6867 |test_ser_loss: 0.9466 |test_accuracy: 0.6070
test: 
 [[117  27  24   2]
 [ 45 225 138  27]
 [ 28  58 261  37]
 [ 13  22  62 143]]
             precision    recall  f1-score   support

          0       0.58      0.69      0.63       170
          1       0.68      0.52      0.59       435
          2       0.54      0.68      0.60       384
          3       0.68      0.60      0.64       240

avg / total       0.62      0.61      0.61      1229


--layer 11--
Epoch: 98 |train_ser_loss: 1.1200 |train_accuracy: 0.5060 |test_ser_loss: 1.1263 |test_accuracy: 0.4809
test: 
 [[ 90  24  42  14]
 [ 52 159 160  64]
 [ 40  87 184  73]
 [ 12   7  63 158]]
             precision    recall  f1-score   support

          0       0.46      0.53      0.49       170
          1       0.57      0.37      0.45       435
          2       0.41      0.48      0.44       384
          3       0.51      0.66      0.58       240

avg / total       0.50      0.48      0.48      1229

--layer 12--
Epoch: 98 |train_ser_loss: 1.0125 |train_accuracy: 0.5074 |test_ser_loss: 1.1512 |test_accuracy: 0.4679
test: 
 [[ 96  37  30   7]
 [ 60 203 120  52]
 [ 64 120 134  66]
 [ 17  22  59 142]]
             precision    recall  f1-score   support

          0       0.41      0.56      0.47       170
          1       0.53      0.47      0.50       435
          2       0.39      0.35      0.37       384
          3       0.53      0.59      0.56       240

avg / total       0.47      0.47      0.47      1229